## 텐서플로우를 이용한 선형 회귀

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from IPython.display import clear_output, Image, display, HTML

In [2]:
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

## 데이터 셋 가져오기

데이터 셋으로는 캘리포니아의 구역별 집 값에 대한 데이터셋을 이용해보도록 한다.

In [3]:
# California housing dataset
# 8개의 Attribute로 이루어진 데이터셋으로, 아래와 같이 구성되어 있다.
housing = fetch_california_housing()

housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

| attribute | 설명 |
| --- | --- |
| MedInc | 구역 내 소득 중간값 |
| HouseAge | 구역 내 나이 중간값 |
| AveRooms | 구역 내 평균 방 갯수 |
| AveBedrms | 구역 내 평균 침실 갯수 |
| Population | 구역 내 인구수 |
| AveOccup | 구역 내 평균 주택점유 |
| Latitude | 구역 위도 |
| Longitude | 구역 경도 |

In [4]:
# 데이터 셋은 총 20640 행(data 수), 8 열(feature 수)로 이루어져 있다.
print(housing.data.shape)

(20640, 8)


## 선형회귀 모델 구성하기

선형 회귀 수식은 기본적으로 아래와 같은 수식으로 이루어져 있다.

$$\hat{y} = b + \theta_1x_1+\theta_2x_2+....+\theta_nx_n$$

|변수| 의미 |
|---|---|
|$\hat{y}$| 예측값 |
|$y$ | 참값 |
| $b$ | 편향(bias) |
| $\theta$ | 가중치(Weight) |


In [5]:
n_epochs = 1000
learning_rate = 0.0000001
rows, cols = housing.data.shape

# 선형회귀 계산의 편의를 위해 bias를 미리 붙임
housing_data = housing.data
bias = np.ones((rows,1))
housing_data_with_bias = np.concatenate([housing_data,bias],axis=1)

# 우리가 구하고자 하는 집값 
target = housing.target.reshape(-1,1)

tf.reset_default_graph() # graph 초기화하기

graph = tf.Graph()
with graph.as_default():
    theta_init = tf.random_uniform([cols+1,1],-1.0,1.0, name='theta_init')
    
    X = tf.placeholder(np.float32,shape=(rows, cols+1), name='x')
    y = tf.placeholder(np.float32,shape=(rows, 1) , name='y')

    theta = tf.Variable(theta_init,name='theta')
    y_pred = tf.matmul(X, theta, name='predictions')
    
    # 차이를 줄이는 방향으로 학습
    error = tf.subtract(y_pred, y, name='error')
    rmse = tf.reduce_mean(tf.square(error),name='mse')

    gradients = 2/rows * tf.matmul(tf.transpose(X), error)
    training_op = tf.assign(theta, theta - learning_rate * gradients) # 오차를 줄이는 방향으로 theta값이 갱신
        
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        for epoch in range(n_epochs):
            feed_dict = {X:housing_data_with_bias,
                         y:target}
            sess.run(training_op,feed_dict=feed_dict)
            if epoch % 100 == 0:
                loss = rmse.eval(feed_dict=feed_dict)
                print("{:3d}th --- {}".format(epoch,loss))
        

  0th --- 5227.05126953125
100th --- 1974.2197265625
200th --- 1524.451416015625
300th --- 1177.7357177734375
400th --- 910.4594116210938
500th --- 704.4197387695312
600th --- 545.5853881835938
700th --- 423.1394348144531
800th --- 328.7438049316406
900th --- 255.9715576171875



## 그래프 시각화하기

In [6]:
show_graph(graph.as_graph_def())